# <b>Probabilidade

## Teorema de Bayes

O Teorema de Bayes descreve a probabilidade de um evento, baseada em informações prévias que podem estar relacionadas ao evento.

A fórmula do Teorema de Bayes é:

$$
P(A|B) = \frac{P(B|A) \cdot P(A)}{P(B)}
$$

Onde:

- $P(A|B)$: probabilidade de $A$ dado que $B$ ocorreu (probabilidade posterior);
- $P(B|A)$: probabilidade de $B$ dado que $A$ ocorreu (verossimilhança);
- $P(A)$: probabilidade de $A$ (probabilidade a priori);
- $P(B)$: probabilidade de $B$ (evidência total).

---

Se houver vários eventos $A_i$, a evidência $P(B)$ pode ser calculada como:

$$
P(B) = \sum_{i=1}^{n} P(B|A_i) \cdot P(A_i)
$$

Essa fórmula permite aplicar o teorema em contextos com múltiplas hipóteses.


In [17]:
def bayes_theorem(likelihood, prior, evidence):
    """
    Calcula a probabilidade posterior usando o Teorema de Bayes.
    
    Parâmetros:
    - likelihood: P(B|A), verossimilhança
    - prior: P(A), probabilidade a priori
    - evidence: P(B), evidência

    Retorna:
    - posterior: P(A|B), probabilidade posterior
    """
    if evidence == 0:
        raise ValueError("A evidência não pode ser zero.")
    
    posterior = (likelihood * prior) / evidence
    return posterior

Exemplo de uso do Teorema de Bayes:<br>
1% da população tem uma doença.<br>
O teste dá positivo em 99% dos casos com a doença e positivo em 5% dos casos sem a doença.

In [19]:
P_A = 0.01              # Probabilidade de ter a doença
P_B_given_A = 0.99      # Probabilidade do teste ser positivo dado que a pessoa tem a doença
P_B_given_not_A = 0.05  # Probabilidade do teste ser positivo dado que a pessoa NÃO tem a doença
P_not_A = 1 - P_A

# Evidência total
P_B = (P_B_given_A * P_A) + (P_B_given_not_A * P_not_A)

# Aplicando o Teorema de Bayes
posterior = bayes_theorem(P_B_given_A, P_A, P_B)
print(f"Probabilidade de ter a doença dado teste positivo: {posterior:.4f}")

Probabilidade de ter a doença dado teste positivo: 0.1667
